In [1]:
## Importing Libraries:

import numpy as np 
import pandas as pd 
import os
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import datetime
import time
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from sklearn import preprocessing

In [2]:
train = pd.read_csv("C:/Users/Aditya/Desktop/BigMart/train.csv")
test = pd.read_csv("C:/Users/Aditya/Desktop/BigMart/test.csv")
sub_df = pd.read_csv("C:/Users/Aditya/Desktop/BigMart/sample_submission.csv",usecols=['Item_Identifier','Outlet_Identifier'])

In [ ]:
train.sample(5)

In [ ]:
train['Age'] = 2020 - train['Outlet_Establishment_Year']
test['Age'] = 2020 - test['Outlet_Establishment_Year']

In [ ]:
train.shape

In [ ]:
train.drop_duplicates()

In [ ]:
test['Item_Outlet_Sales'] = 0

In [ ]:
df = pd.concat([train,test], ignore_index = True)

In [ ]:
df

In [ ]:
train.hist(bins=50, figsize=(15, 15))

In [ ]:
test.hist(bins=50, figsize=(15, 15))

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
sns.boxplot(np.log(train['Item_Visibility']))

In [ ]:
train['Item_Visibility_log'] = np.log(train['Item_Visibility'])

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.skew()

In [ ]:
test.skew()

In [3]:
train['Item_Identifier_Outlet_Identifier'] = train['Item_Identifier'].astype(str)+'_'+train['Outlet_Identifier']
test['Item_Identifier_Outlet_Identifier'] = test['Item_Identifier'].astype(str)+'_'+test['Outlet_Identifier']

In [4]:
df['Item_Identifier_Outlet_Identifier'] = df['Item_Identifier'].astype(str)+'_'+df['Outlet_Identifier']

NameError: name 'df' is not defined

In [5]:
train['Item_Fat_Content'] = np.where(train['Item_Fat_Content'] == 'Low Fat',0,
                            np.where(train['Item_Fat_Content'] == 'LF',0,
                            np.where(train['Item_Fat_Content'] == 'low fat',0,1)))

test['Item_Fat_Content'] = np.where(test['Item_Fat_Content'] == 'Low Fat',0,
                           np.where(test['Item_Fat_Content'] == 'LF',0,
                           np.where(test['Item_Fat_Content'] == 'low fat',0,1)))

In [ ]:
df['Item_Fat_Content'] = np.where(df['Item_Fat_Content'] == 'Low Fat',0,
                           np.where(df['Item_Fat_Content'] == 'LF',0,
                           np.where(df['Item_Fat_Content'] == 'low fat',0,1)))

In [ ]:
test['Item_Outlet_Sales'] = 0

In [6]:
train['Item_Weight'].describe()

count    7060.000000
mean       12.857645
std         4.643456
min         4.555000
25%         8.773750
50%        12.600000
75%        16.850000
max        21.350000
Name: Item_Weight, dtype: float64

In [7]:
test['Item_Weight'].describe()

count    4705.000000
mean       12.695633
std         4.664849
min         4.555000
25%         8.645000
50%        12.500000
75%        16.700000
max        21.350000
Name: Item_Weight, dtype: float64

In [ ]:
test.describe()

In [ ]:
df.isnull().sum()

In [8]:
### Missing values:

train['Item_Weight'] = train.groupby('Item_Identifier').transform(lambda x: x.fillna(x.mean()))
test['Item_Weight'] = test.groupby('Item_Identifier').transform(lambda x: x.fillna(x.mean()))

In [ ]:
df['Item_Weight'] = df.groupby('Item_Identifier').transform(lambda x: x.fillna(x.mean()))

In [9]:
### Missing values:

train['Item_Weight'] = train['Item_Weight'].fillna((train['Item_Weight'].mean()))
test['Item_Weight'] = test['Item_Weight'].fillna((test['Item_Weight'].mean()))

In [ ]:
print(train.shape, test.shape)

In [ ]:
train.isnull().sum()

In [ ]:
df['Outlet_Size'] = df['Outlet_Size'].fillna('Large')

In [ ]:
train['Outlet_Size'].value_counts()

In [ ]:
train['Type_MRP_total'] = train.groupby(['Item_Type'])['Item_MRP'].transform('sum')
#train['Type_MRP_mean'] = train.groupby(['Item_Type'])['Item_MRP'].transform('mean')

train['Out_Iden_MRP_total'] = train.groupby(['Outlet_Identifier'])['Item_MRP'].transform('sum')
#train['Out_Iden_MRP_mean'] = train.groupby(['Outlet_Identifier'])['Item_MRP'].transform('mean')

train['Year_MRP_total'] = train.groupby(['Outlet_Establishment_Year'])['Item_MRP'].transform('sum')
#train['Year_MRP_mean'] = train.groupby(['Outlet_Establishment_Year'])['Item_MRP'].transform('mean')

train['Out_Type_MRP_total'] = train.groupby(['Outlet_Type'])['Item_MRP'].transform('sum')
#train['Out_Type_MRP_mean'] = train.groupby(['Outlet_Type'])['Item_MRP'].transform('mean')

train['Item_Identifiere_MRP_total'] = train.groupby(['Item_Identifier'])['Item_MRP'].transform('sum')


In [ ]:
train.sample(5)

In [ ]:
test['Type_MRP_total'] = test.groupby(['Item_Type'])['Item_MRP'].transform('sum')
#test['Type_MRP_mean'] = test.groupby(['Item_Type'])['Item_MRP'].transform('mean')

test['Out_Iden_MRP_total'] = test.groupby(['Outlet_Identifier'])['Item_MRP'].transform('sum')
#test['Out_Iden_MRP_mean'] = test.groupby(['Outlet_Identifier'])['Item_MRP'].transform('mean')

test['Year_MRP_total'] = test.groupby(['Outlet_Establishment_Year'])['Item_MRP'].transform('sum')
#test['Year_MRP_mean'] = test.groupby(['Outlet_Establishment_Year'])['Item_MRP'].transform('mean')

test['Out_Type_MRP_total'] = test.groupby(['Outlet_Type'])['Item_MRP'].transform('sum')
#test['Out_Type_MRP_mean'] = test.groupby(['Outlet_Type'])['Item_MRP'].transform('mean')


test['Item_Identifiere_MRP_total'] = test.groupby(['Item_Identifier'])['Item_MRP'].transform('sum')


In [10]:
train.sample(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Identifier_Outlet_Identifier
4835,DRC49,8.670,0,0.065120,Soft Drinks,145.7128,OUT027,1985,Medium,Tier 3,Supermarket Type3,4026.7584,DRC49_OUT027
5774,FDZ26,11.600,1,0.144241,Dairy,239.4222,OUT049,1999,Medium,Tier 1,Supermarket Type1,3346.3108,FDZ26_OUT049
2169,DRG48,5.780,0,0.014555,Soft Drinks,145.2102,OUT046,1997,Small,Tier 1,Supermarket Type1,3062.0142,DRG48_OUT046
4217,DRK59,8.895,0,0.000000,Hard Drinks,232.9616,OUT045,2002,NaN,Tier 2,Supermarket Type1,2812.3392,DRK59_OUT045
2159,FDH35,18.250,0,0.100844,Starchy Foods,166.3526,OUT010,1998,NaN,Tier 3,Grocery Store,164.4526,FDH35_OUT010


In [ ]:
## One hot Encoding:
dummy = pd.get_dummies(train['Item_Type'])
dummy1 = pd.get_dummies(test['Item_Type'])

In [ ]:
train = train.merge(dummy,left_index = True, right_index = True)
test = test.merge(dummy1,left_index = True, right_index = True)

In [ ]:
item_count = train['Item_Type'].value_counts().to_dict()
item_count1 = test['Item_Type'].value_counts().to_dict()

In [ ]:
train['Item_Type'] = train['Item_Type'].map(item_count)
test['Item_Type'] = test['Item_Type'].map(item_count1)

In [ ]:
outlet_count = train['Outlet_Identifier'].value_counts().to_dict()
outlet_count1 = test['Outlet_Identifier'].value_counts().to_dict()


train['Outlet_Identifier'] = train['Outlet_Identifier'].map(outlet_count)
test['Outlet_Identifier'] = test['Outlet_Identifier'].map(outlet_count1)

In [ ]:
item_count = train['Item_Identifier'].value_counts().to_dict()
item_count1 = test['Item_Identifier'].value_counts().to_dict()


train['Item_Identifier'] = train['Item_Identifier'].map(item_count)
test['Item_Identifier'] = test['Item_Identifier'].map(item_count)

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
train.head()

In [ ]:
## One hot Encoding:
dummy = pd.get_dummies(train['Outlet_Type'])
dummy1 = pd.get_dummies(test['Outlet_Type'])

In [ ]:
train = train.merge(dummy,left_index = True, right_index = True)
test = test.merge(dummy1,left_index = True, right_index = True)

In [ ]:
## One hot Encoding:
dummy_loc = pd.get_dummies(train['Outlet_Location_Type'])
dummy1_loc = pd.get_dummies(test['Outlet_Location_Type'])

In [ ]:
train = train.merge(dummy_loc,left_index = True, right_index = True)
test = test.merge(dummy1_loc,left_index = True, right_index = True)

In [ ]:
train['Outlet_Establishment_Year'] = train['Outlet_Establishment_Year'].map({
        1985: 0,1987: 1,1997: 2,1998: 3,1999: 4,2002: 5,2004: 6,2007: 7,2009: 8 })


test['Outlet_Establishment_Year'] = test['Outlet_Establishment_Year'].map({
        1985: 0,1987: 1,1997: 2,1998: 3,1999: 4,2002: 5,2004: 6,2007: 7,2009: 8 })


In [ ]:
train['Outlet_Size'].value_counts()

In [ ]:
### Missing Values:
train['Outlet_Size'] = train['Outlet_Size'].fillna('Large')
test['Outlet_Size'] = test['Outlet_Size'].fillna('Large')

In [ ]:
train['Outlet_Size'] = train['Outlet_Size'].map({'Large':0, 'Small':1,'Medium':2,'High':3})
test['Outlet_Size'] = test['Outlet_Size'].map({'Large':0, 'Small':1,'Medium':2,'High':3})

In [ ]:
train.head()

In [ ]:
train.describe().T

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(train.corr(),annot=True,fmt=".2f")

In [ ]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(train, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = train.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(train, 0.75)
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
sns.boxplot(train['Item_Visibility'])

In [ ]:
sns.boxplot(test['Item_Visibility'])

In [ ]:
#Removing Outlires
from scipy.stats import zscore

z_scores = zscore(X)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_df = X[filtered_entries]
train_df = new_df

In [ ]:
cols_to_norm = ['Item_Visibility']
train['Item_Visibility_1'] = train[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
test['Item_Visibility_1'] = test[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
train.sample(5)

In [ ]:
sns.distplot(train['Item_Visibility_1'])

In [ ]:
train.head()

In [ ]:
train.drop(['Outlet_Location_Type','Outlet_Type'], axis = 1, inplace = True)
test.drop(['Outlet_Location_Type','Outlet_Type'], axis = 1, inplace = True)

##,'Outlet_Size','Outlet_Identifier','Item_Identifier','Today','Item_Type'

In [ ]:
train.head()

In [ ]:
corr_features

In [ ]:
Y = train['Item_Outlet_Sales']
X = train.drop(['Item_Outlet_Sales','Item_Identifier_Outlet_Identifier','Outlet_Establishment_Year','Item_Fat_Content',
               'Item_Identifier','Outlet_Identifier','Item_Visibility','Item_Weight'], axis=1)
                
####'Item_Visibility','Item_Weight'                

In [ ]:
test['Item_Outlet_Sales'] = 0
y_test = test[['Item_Outlet_Sales']]

In [ ]:
test_df = test.drop(['Item_Outlet_Sales','Item_Identifier_Outlet_Identifier','Outlet_Establishment_Year','Item_Fat_Content',
                    'Item_Identifier','Outlet_Identifier','Item_Visibility','Item_Weight'], axis=1)

In [ ]:
X_test_norm.shape

In [ ]:
#REgressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
#Evalution Metrix
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

In [ ]:
gbr = GradientBoostingRegressor(random_state=0)
xgb = xgboost.XGBRegressor(n_jobs=-1)
et = ExtraTreesRegressor(n_jobs=-1)
rf = RandomForestRegressor(n_jobs=-1)
ds = DecisionTreeRegressor()
cat = CatBoostRegressor()
knn = KNeighborsRegressor()
lgb = LGBMRegressor()
lr = LinearRegression()


In [ ]:
reg = {
"LinearRegression": LinearRegression(),
"KNeighborsRegressor":KNeighborsRegressor(n_neighbors=2),
"AdaBoostRegressor":AdaBoostRegressor(random_state=0, n_estimators=100),
"LGBMRegressor":LGBMRegressor(),
"Ridge": Ridge(alpha=1.0),
"ElasticNet":ElasticNet(random_state=0),
"GradientBoostingRegressor":GradientBoostingRegressor(random_state=0),
"DecisionTreeRegressor": DecisionTreeRegressor(),
"ExtraTreesRegressor": ExtraTreesRegressor(n_jobs=-1),
"RandomForestRegressor": RandomForestRegressor(n_jobs=-1),
"XGBRegressor":xgboost.XGBRegressor(n_jobs=-1),
"CatBoostRegressor":CatBoostRegressor(iterations=900, depth=5, learning_rate=0.05,loss_function = 'RMSE')
#"VottingRegressor": VotingRegressor([('rf', rf),('xgb',xgb),('et',et)],n_jobs=-1)
#"Stcking" : StackingRegressor(estimators=[('et', et),('xgb',xgb),('rf',rf)], final_estimator=LinearRegression(), cv=5)
}

In [ ]:
%%time
dic =  {"Model":[],"R2_Train":[],"RMSE_Train":[]}
for name, model in reg.items():
    if name == 'CatBoostRegressor':
        model.fit(X, Y,verbose=False)
        #model.fit(X_train_norm, Y,verbose=False)
    else:
        model.fit(X, Y)
        #model.fit(X_train_norm, Y)
    #y_test_pre = model.predict(test_df)
    y_train_pre = model.predict(X)
    #y_train_pre = model.predict(X_train_norm)
    r2_train = r2_score(Y, y_train_pre)
    rmse_train  = np.sqrt(mean_squared_error(Y, y_train_pre))
    print("--------------------------------------------------------------")
    print("Model:", name)
    print("-----Training Data Evaluation-----")
    print("R2 Value: ", r2_score(Y, y_train_pre))
    print("RMSE: ",np.sqrt(mean_squared_error(Y, y_train_pre)))
    dic["Model"].append(name)
    dic["R2_Train"].append(r2_train)
    dic["RMSE_Train"].append(rmse_train)
#final_data = pd.DataFrame(dic)
#print("==================================================================")
#print("Best Model")
#print("==================================================================")
#print(final_data.max())
#print("==================================================================")
#final_data.sort_values("RMSE_Train", axis = 0, ascending = True)

In [ ]:
final_data = pd.DataFrame(dic)
final_data

In [ ]:
final_data.sort_values("RMSE_Train", axis = 0, ascending = True)

In [ ]:
%%time

model = VotingRegressor([('rf',rf),('xgb',xgb),('knn',knn),('lgb',lgb),('cat',cat),('ds',ds)],n_jobs=-1)
#model = StackingRegressor([('rf',rf),('xgb',xgb),('knn',knn),('lgb',lgb),('cat',cat),('ds',ds)],n_jobs=-1)

###('rf',rf),('xgb',xgb),('lgb',lgb),('cat',cat)

model.fit(X, Y)
#model.fit(X_train_norm, Y)

#y_test_pre = model.predict(test_df)
#r2_train = r2_score(Y, y_train_pre)
#rmse_train  = np.sqrt(mean_squared_error(Y, y_train_pre))
#print("Model:", name)
#print("-----Training Data Evalution-----")
#print("R2 Value: ", r2_score(Y, y_train_pre))
#print("RMSE: ",np.sqrt(mean_squared_error(Y, y_train_pre)))


y_test_pre = model.predict(test_df)
#y_test_pre = model.predict(X_test_norm)

r2_test = r2_score(y_test, y_test_pre)
rmse_test  = np.sqrt(mean_squared_error(y_test, y_test_pre))
#print("Model:", name)
print("-----Training Data Evalution-----")
print("R2 Value: ", r2_score(y_test, y_test_pre))
print("RMSE: ",np.sqrt(mean_squared_error(y_test, y_test_pre)))

In [ ]:
test_pre_df = {'Item_Outlet_Sales': y_test_pre}
test_pre_df = pd.DataFrame(test_pre_df)

In [ ]:
test_pre_df.describe()

In [ ]:
dataset  = pd.concat([sub_df,test_pre_df], axis = 1)

In [ ]:
sns.distplot(dataset['Item_Outlet_Sales'])

In [ ]:
dataset.to_csv("C:/Users/Aditya/Desktop/BigMart/vot_xgb_rf_knn_cat3.csv",index=False)